In [2]:
import numpy as np
import pandas as pd

In [3]:
email_df = pd.read_csv('data-2024-07-16.csv', sep=';')

In [4]:
email_df.columns

Index(['Message ID', 'Received At', 'Processed At', 'Date', 'Trip Locator',
       'Customer', 'Trip Start', 'Trip End', 'Trip Start Date/time',
       'Trip End Date/time', 'Routing', 'Passengers', 'One Way', 'Source',
       'Aircraft', 'Source Suggested Price', 'Source Suggested Price.1',
       'Source Suggested Price.2', 'Source Suggested Price.3',
       'Source Suggested Price.4', 'Phone', 'Mobile', 'Email Subject'],
      dtype='object')

In [5]:
df = email_df[['Trip Locator', 'Customer', 'Trip Start Date/time', 'Trip End Date/time', 'Routing']]

In [6]:
df = df.dropna(subset=['Routing'], inplace = False)

In [7]:
df.head()

,Trip Locator,Customer,Trip Start Date/time,Trip End Date/time,Routing
0,DOKH8W,"Grant Casstevens (grant@dupageaerospace.com, 6...",07/16/2024 @ 03:30pm departure,NaN,BZN - SLC
1,Y62XE9,"Air Partner (nevin.jayawardena@airpartner.com,...",07/22/2024 @ 10:00am departure,07/28/2024 @ 09:00am departure,SDL - GEG - SDL
2,V6KM2F,Hunter Wrather (h.wrather@trilogyaviationgroup...,07/16/2024 @ 09:00am departure,07/19/2024 @ 05:00pm departure,FAR - MSP - MWCR - FAR
3,WAIWT1,"Wil Jordan (wjordan@luxury.aero, 631-676-7488)",07/24/2024 @ 10:00am departure,07/25/2024 @ 06:00pm departure,BCT - HOU - BCT
4,M4WZRZ,Nikita Volzhenin (n.volzhenin@primesk.kz),07/16/2024 @ 09:00am departure,NaN,SFO - MIA


In [8]:
df['Routing'] = df['Routing'].str.split(' - ')

In [9]:
df_filtered = df[df['Routing'].apply(lambda x: len(x) <= 3)].reset_index().drop(['index'], axis=1)

In [10]:
def route():
    origin_array = []
    destination_array = []
    return_array = []
    for i in df_filtered['Routing']:
        if len(i[0]) < 4:
            origin_array.append('K' + i[0])
        else:
            origin_array.append(i[0])
        if len(i[1]) < 4:
            destination_array.append('K' + i[1])
        else:
            destination_array.append(i[1])
        if len(i) > 2:
            if len(i[2]) < 4:
                return_array.append('K' + i[2])
            else:
                return_array.append(i[2])
        else:
            return_array.append(None)
    origin_df = pd.DataFrame(origin_array, columns =['Origin'])
    destination_df = pd.DataFrame(destination_array, columns = ['Destination'])
    return_df = pd.DataFrame(return_array, columns = ['Return'])
    route_df = origin_df.join(destination_df)
    route_df = route_df.join(return_df)
    return route_df

route = route()

In [11]:
requests_df_cleaned = df_filtered.join(route)

In [12]:
requests_df_cleaned.head(10)

,Trip Locator,Customer,Trip Start Date/time,Trip End Date/time,Routing,Origin,Destination,Return
0,DOKH8W,"Grant Casstevens (grant@dupageaerospace.com, 6...",07/16/2024 @ 03:30pm departure,NaN,"[BZN, SLC]",KBZN,KSLC,None
1,Y62XE9,"Air Partner (nevin.jayawardena@airpartner.com,...",07/22/2024 @ 10:00am departure,07/28/2024 @ 09:00am departure,"[SDL, GEG, SDL]",KSDL,KGEG,KSDL
2,WAIWT1,"Wil Jordan (wjordan@luxury.aero, 631-676-7488)",07/24/2024 @ 10:00am departure,07/25/2024 @ 06:00pm departure,"[BCT, HOU, BCT]",KBCT,KHOU,KBCT
3,M4WZRZ,Nikita Volzhenin (n.volzhenin@primesk.kz),07/16/2024 @ 09:00am departure,NaN,"[SFO, MIA]",KSFO,KMIA,None
4,UFIBD3,"Kobina Brew-Hammond (kobina@saharajets.com, 38...",07/19/2024 @ 07:00am departure,NaN,"[DFW, SFO]",KDFW,KSFO,None
5,GSLJUX,"Charter Team (charter@jet79.com, +1 (310) 888-...",08/25/2024 @ 10:00am departure,NaN,"[VNY, OPF]",KVNY,KOPF,None
6,TFGP8Y,"Jordan Brown (davido@flycfg.com, 888-634-7449)",08/03/2024 @ 10:00am departure,NaN,"[VNY, HNL]",KVNY,KHNL,None
7,DS11U6,Jonathan Scott (jonathan.scott@aircharterservi...,07/16/2024 @ 05:00pm departure,NaN,"[FXE, MMU]",KFXE,KMMU,None
8,RQSODM,"Thomas Nelson (tyler@trilogyaviationgroup.com,...",07/17/2024 @ 10:00am departure,NaN,"[FTW, MSP]",KFTW,KMSP,None
9,CBZITD,"Kaitlyn Wheatley (kwheatley@flyxo.com, +1 888 ...",07/20/2024 @ 09:00am departure,07/20/2024 @ 03:30pm departure,"[MVY, PWM, MVY]",KMVY,KPWM,KMVY


In [13]:
airport_df = pd.read_csv('cleaned_airport.csv', sep=',')

In [14]:
airport_df.head(10)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,region_name,local_region
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,California,CA
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,680.0,Illinois,IL
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13.0,New York,NY
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,Georgia,GA
4,3878,KSFO,large_airport,San Francisco International Airport,37.619806,-122.374821,13.0,California,CA
5,3521,KEWR,large_airport,Newark Liberty International Airport,40.692501,-74.168701,18.0,New Jersey,NJ
6,3488,KDFW,large_airport,Dallas Fort Worth International Airport,32.896801,-97.038002,607.0,Texas,TX
7,3631,KLAS,large_airport,Harry Reid International Airport,36.083361,-115.151817,2181.0,Nevada,NV
8,3670,KMCO,large_airport,Orlando International Airport,28.429399,-81.308998,96.0,Florida,FL
9,3486,KDEN,large_airport,Denver International Airport,39.861698,-104.672997,5431.0,Colorado,CO


In [15]:
filtered_airport_df = airport_df[['ident', 'latitude_deg', 'longitude_deg','region_name']]
filtered_airport_df.head()

,ident,latitude_deg,longitude_deg,region_name
0,KLAX,33.942501,-118.407997,California
1,KORD,41.978600,-87.904800,Illinois
2,KJFK,40.639447,-73.779317,New York
3,KATL,33.636700,-84.428101,Georgia
4,KSFO,37.619806,-122.374821,California


In [29]:
requests_df_cleaned = requests_df_cleaned.set_index('Origin').join(filtered_airport_df.set_index('ident'), how = 'left')

KeyError: "None of ['Origin'] are in the columns"

In [33]:
requests_df_cleaned = requests_df_cleaned.reset_index()

In [34]:
requests_df_cleaned.head()

,index,Origin,Trip Locator,Customer,Trip Start Date/time,Trip End Date/time,Routing,Destination,Return,latitude_deg,longitude_deg,region_name
0,0,KBZN,DOKH8W,"Grant Casstevens (grant@dupageaerospace.com, 6...",07/16/2024 @ 03:30pm departure,NaN,"[BZN, SLC]",KSLC,None,45.777500,-111.153000,Montana
1,1,KSDL,Y62XE9,"Air Partner (nevin.jayawardena@airpartner.com,...",07/22/2024 @ 10:00am departure,07/28/2024 @ 09:00am departure,"[SDL, GEG, SDL]",KGEG,KSDL,33.622898,-111.911003,Arizona
2,2,KBCT,WAIWT1,"Wil Jordan (wjordan@luxury.aero, 631-676-7488)",07/24/2024 @ 10:00am departure,07/25/2024 @ 06:00pm departure,"[BCT, HOU, BCT]",KHOU,KBCT,26.378500,-80.107697,Florida
3,3,KSFO,M4WZRZ,Nikita Volzhenin (n.volzhenin@primesk.kz),07/16/2024 @ 09:00am departure,NaN,"[SFO, MIA]",KMIA,None,37.619806,-122.374821,California
4,4,KDFW,UFIBD3,"Kobina Brew-Hammond (kobina@saharajets.com, 38...",07/19/2024 @ 07:00am departure,NaN,"[DFW, SFO]",KSFO,None,32.896801,-97.038002,Texas
